In [2]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np




In [49]:
conninput = sqlite3.connect("intermediate2.db")

In [16]:
from sklearn.model_selection import cross_val_predict, GridSearchCV, cross_val_score 

In [18]:
from sklearn.ensemble import RandomForestRegressor

In [20]:
from sklearn.feature_selection import SelectFromModel

In [22]:
from sklearn import metrics

In [47]:
def randomforestandreturn(year, conn):
    
    raw = pd.read_sql_query("SELECT * FROM dataset"+str(year), conn)
    dataset_X = pd.concat([raw.iloc[:,5:-4],raw.iloc[:,-2:]], axis = 1)
    dataset_y = raw.iloc[:,2]

    # Perform Grid-Search 
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,20),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(dataset_X, dataset_y)
    best_params = grid_result.best_params_

    sel = SelectFromModel(RandomForestRegressor(max_depth=best_params["max_depth"],
                                n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False))
    sel.fit(dataset_X, dataset_y)

    rfr = RandomForestRegressor(max_depth=best_params["max_depth"],
                                n_estimators=best_params["n_estimators"],
                                random_state=False, verbose=False)
    # Perform K-Fold CV
    scores = cross_val_score(rfr, dataset_X, dataset_y, cv=10, scoring='neg_mean_absolute_error')
    predictions = cross_val_predict(rfr, dataset_X,dataset_y, cv=10)
    MSE = metrics.mean_squared_error(dataset_y, predictions)

    #    return predictions
    
    rfr.fit(dataset_X, dataset_y)
    
    return MSE, pd.DataFrame(rfr.feature_importances_, index = dataset_X.columns, columns = [year]).T

In [45]:
connoutput =  sqlite3.connect("gini.db")

In [52]:
for startingyear in range(1090,1091,10):
    mse, gini = randomforestandreturn(startingyear, conninput)
    print (str(startingyear)+' '+ str(mse))
    gini.to_sql('gini'+str(startingyear), con=connoutput, if_exists='replace')

DatabaseError: Execution failed on sql 'SELECT * FROM dataset1090': no such table: dataset1090

In [69]:
msedict ={ 
960: 4.082307549722228,
970: 3.5783551861319176,
980: 3.6326095990712735,
990: 3.6633516652291878,
1000: 3.8052264096572643,
1010: 3.6143887806676966,
1020: 3.560617868522442,
1030: 4.500202585385132,
1040: 4.054315647657062,
1050: 4.412586341145719,
1060: 4.0499107646002335,
1070: 3.7732951682251143,
1080: 3.461161422872546,
1100: 3.868290170407143,
1110: 4.851939527761255,
1120: 4.5343699502459,
1130: 3.9520915803852303,
1140: 4.069068135485917,
1150: 4.06430410596036,
1160: 4.915905976369748,
1170: 5.163614328177442,
1180: 4.336374185617349,
1190: 5.505146077033698,
1200: 4.7643094513658175,
1210: 4.620173880598139,
1220: 5.212462547258111,
1230: 4.944277216055614,
1240: 4.918187905096115,
1250: 5.910373351127136,
1260: 6.068559620813858,
1270: 5.383539200836642,
1280: 6.007526107458532
}

In [66]:
dflist = []

for startingyear in range(960,1281,10):
    if startingyear == 1090:
        continue
    df = pd.read_sql_query("SELECT * FROM gini"+str(startingyear), connoutput)
    df.iloc[:,0] = startingyear
        
    dflist.append(df)

In [67]:
resultdf = pd.concat(dflist)

In [88]:
resultdf

,year,Lindex,c_index_year,c_birthyear,c_deathyear,c_death_age,c_fl_earliest_year,c_fl_ey_nh_code,c_fl_latest_year,Longitude,...,entry_163,entry_165,entry_166,entry_168,entry_169,entry_225,entry_226,entry_248,status,nemesis
0,960,0.024782,0.007595,0.016037,0.135908,0.187089,0.154292,0.032821,0.145775,0.054785,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.043025,0.009512
0,970,0.016413,0.030780,0.022172,0.072175,0.087275,0.020076,0.009959,0.017156,0.244544,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.111045,0.003666
0,980,0.015187,0.044720,0.009437,0.011213,0.336195,0.013623,0.011125,0.020498,0.252130,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.109942,0.011767
0,990,0.010015,0.014796,0.065225,0.098400,0.102190,0.002506,0.001675,0.001404,0.082659,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.135849,0.008234
0,1000,0.012356,0.006447,0.015622,0.102042,0.170827,0.001654,0.004010,0.013101,0.058195,...,0.000000,0.001090,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.362278,0.002069
0,1010,0.037951,0.041403,0.014194,0.036800,0.190999,0.008101,0.041298,0.020716,0.062001,...,0.000000,0.004955,0.000000,0.0,0.000000,0.0,0.000000,0.001724,0.247087,0.012268
0,1020,0.018776,0.018133,0.017103,0.198204,0.039710,0.001526,0.004346,0.002161,0.050647,...,0.000000,0.004364,0.000041,0.0,0.000000,0.0,0.000000,0.000000,0.265660,0.017703
0,1030,0.008623,0.021901,0.018202,0.184477,0.197904,0.000934,0.003262,0.003931,0.058063,...,0.000000,0.001640,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.244834,0.049296
0,1040,0.050479,0.036938,0.052688,0.032558,0.072265,0.012982,0.012005,0.026886,0.065285,...,0.000000,0.018870,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.176475,0.056055
0,1050,0.027167,0.021676,0.032203,0.067806,0.121909,0.012974,0.007987,0.006973,0.086197,...,0.000000,0.002914,0.000826,0.0,0.000000,0.0,0.000000,0.000000,0.118665,0.053188


In [87]:
templist = ['year']
for i in range(1, len(resultdf.columns)):
    templist.append(resultdf.columns[i])
resultdf.columns = templist

In [98]:
resultdf.reset_index(drop = True, inplace = True)

In [99]:
resultdf

,year,Lindex,c_index_year,c_birthyear,c_deathyear,c_death_age,c_fl_earliest_year,c_fl_ey_nh_code,c_fl_latest_year,Longitude,...,entry_163,entry_165,entry_166,entry_168,entry_169,entry_225,entry_226,entry_248,status,nemesis
0,960,0.024782,0.007595,0.016037,0.135908,0.187089,0.154292,0.032821,0.145775,0.054785,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.043025,0.009512
1,970,0.016413,0.030780,0.022172,0.072175,0.087275,0.020076,0.009959,0.017156,0.244544,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.111045,0.003666
2,980,0.015187,0.044720,0.009437,0.011213,0.336195,0.013623,0.011125,0.020498,0.252130,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.109942,0.011767
3,990,0.010015,0.014796,0.065225,0.098400,0.102190,0.002506,0.001675,0.001404,0.082659,...,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.135849,0.008234
4,1000,0.012356,0.006447,0.015622,0.102042,0.170827,0.001654,0.004010,0.013101,0.058195,...,0.000000,0.001090,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.362278,0.002069
5,1010,0.037951,0.041403,0.014194,0.036800,0.190999,0.008101,0.041298,0.020716,0.062001,...,0.000000,0.004955,0.000000,0.0,0.000000,0.0,0.000000,0.001724,0.247087,0.012268
6,1020,0.018776,0.018133,0.017103,0.198204,0.039710,0.001526,0.004346,0.002161,0.050647,...,0.000000,0.004364,0.000041,0.0,0.000000,0.0,0.000000,0.000000,0.265660,0.017703
7,1030,0.008623,0.021901,0.018202,0.184477,0.197904,0.000934,0.003262,0.003931,0.058063,...,0.000000,0.001640,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.244834,0.049296
8,1040,0.050479,0.036938,0.052688,0.032558,0.072265,0.012982,0.012005,0.026886,0.065285,...,0.000000,0.018870,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.176475,0.056055
9,1050,0.027167,0.021676,0.032203,0.067806,0.121909,0.012974,0.007987,0.006973,0.086197,...,0.000000,0.002914,0.000826,0.0,0.000000,0.0,0.000000,0.000000,0.118665,0.053188


In [105]:
msedf = pd.DataFrame(msedict, index = ['mse']).T

In [108]:
msedf.index.name = 'year'

In [111]:
resultdf.join(msedf, on = 'year').to_csv('randomforestresult.csv')

In [91]:
resultdf.join(mse, on = 'year')

ValueError: Joining multiple DataFrames only supported for joining on index